# 使用 Streamlit 與 Ollama 建立聊天介面
## 在 Google Colab 運行 Streamlit 並讓本機存取
Google Colab 預設**無法直接執行 `streamlit run`**，因為 Colab 是一個 **雲端環境**，但我們可以透過 **localtunnel** 或 **ngrok** 來建立一個**可公開存取的網址**，讓本機能夠連上 Colab 內運行的 Streamlit 應用。

### **步驟 1：安裝必要套件**
在 **Colab** 內執行安裝 streamlit：

In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


### **程式簡介**
這支程式使用 **Streamlit** 建立一個簡單的 **單輪聊天介面**，並透過 **Ollama（類 OpenAI API）** 來處理對話請求。它的主要功能是：
1. **使用者輸入訊息**，顯示在聊天框內。
2. **程式將訊息傳送至 Ollama API（本機端 11434 連接埠）** 來獲取 AI 回應。
3. **顯示 AI 回應**，並將對話內容保留在 `st.session_state` 中，讓使用者可以看到完整的對話紀錄。

這是一個簡單的 **本機 AI 助手**，可以用來測試 **Ollama 所管理模型**，或作為更進階應用的基礎。這裡使用 **魔術指令** 在 Colab 中寫入程式：


In [ ]:
%%writefile basic.py
import os
from openai import OpenAI
import streamlit as st

# 設定 OpenAI API 金鑰與模型端點
token = "ollama"
endpoint = "http://localhost:11434/v1/"
model_name = "gemma2:9b"

client = OpenAI(base_url=endpoint, api_key=token)

# 單輪對話初始化的 system prompt
SYSTEM_PROMPT = {"role": "system", "content": "你是一個有用的助手。且會根據使用者輸入的語言做回應。"}

# 初始化對話歷史，僅用於顯示（不保存上下文）
if "display_messages" not in st.session_state:
    st.session_state.display_messages = []

# 顯示對話歷史
for message in st.session_state.display_messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# 接收使用者輸入
if user_input := st.chat_input("請輸入您的訊息..."):
    # 顯示使用者訊息
    st.session_state.display_messages.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.markdown(user_input)

    # 組合單輪對話訊息
    messages = [SYSTEM_PROMPT, {"role": "user", "content": user_input}]

    # 調用 OpenAI API 獲取回應
    response = client.chat.completions.create(
        messages=messages,
        model=model_name,
        temperature=1.0,
        top_p=1.0,
        max_tokens=1000,
    )

    # 獲取助理回應內容
    assistant_response = response.choices[0].message.content

    # 顯示助理回應
    st.session_state.display_messages.append({"role": "assistant", "content": assistant_response})
    with st.chat_message("assistant"):
        st.markdown(assistant_response)



## 在 Colab 運行程式

![](https://github.com/andy6804tw/crazyai-llm/blob/main/docs/2.LLM%E5%AF%A6%E6%88%B0%E6%87%89%E7%94%A8/images/img-ollama-streamlit-1.png)

📌 **指令說明**

- `!streamlit run basic.py &>./logs.txt &` → **在背景運行 Streamlit**，並將日誌輸出到 `logs.txt`
- `npx --yes localtunnel --port 8501` → **開放公開網址**，讓本機存取 Colab 內的 Streamlit
- `curl ipv4.icanhazip.com` → **顯示 Colab 機器的外部 IP**

In [ ]:
!streamlit run basic.py &>./logs.txt & npx --yes localtunnel --port 8501 & curl ipv4.icanhazip.com

此命令會運行 **Streamlit 應用程式 (`basic.py`)**，並將輸出記錄到 `logs.txt`。同時，它使用 **localtunnel** 將應用程式在 **8501 連接埠** 上 **臨時公開到網際網路**。

在 **Colab** 執行該命令後，應該會顯示類似以下的輸出內容：

![](https://github.com/andy6804tw/crazyai-llm/blob/main/docs/2.LLM%E5%AF%A6%E6%88%B0%E6%87%89%E7%94%A8/images/img-ollama-streamlit-3.png)

請**複製 IP 位址**（例如 **35.106.17.127**），然後**點擊產生的 URL**，即可進入應用程式的頁面。

![](https://github.com/andy6804tw/crazyai-llm/blob/main/docs/2.LLM%E5%AF%A6%E6%88%B0%E6%87%89%E7%94%A8/images/img-ollama-streamlit-2.png)
